# Sorting images by their principal colors

In [1]:
import pandas as pd
from PIL import Image
import math
import colorsys
import os

## Loading the datasets

Results may vary using the results obtained with the knee algorithm or restricting the number of clusters (for example, using only three clusters).

In [2]:
df_col = pd.read_csv("ColorCluster_3clusters_RGB.csv")
df_per = pd.read_csv("ColorCluster_3clusters_Percentage.csv")

In [3]:
df_col.head(10)

,mallows,peach2,lamp,mos,pipe,coc,peach,flowers,ic,raccoon,...,6bf7bb6ba078ffb96fa1bf7dc73d28a0,fe7e1edbcda543fc2d7e3dc4bf54c5fe,d072cb092a6b5e3e5e9746ff42a928e3,68788d484729dc0297da343a0afe835e,ac144eb4fe749a4c231bbe874f0dd1ea,a5231680454b32cf7c34d19d7f55bf91,68320e05c6b538567d454c1bfd21f143,28c848151a54b63dd07543b3fcb346ae,350ecf9416b807877ec2acf132e6df28,96a5bda36aee92754d0a50ca2e4024ab
0,"(202, 118, 132)","(143, 51, 45)","(233, 232, 236)","(46, 114, 154)","(160, 128, 90)","(212, 214, 215)","(111, 28, 26)","(63, 40, 21)","(194, 192, 175)","(128, 125, 129)",...,"(104, 104, 98)","(7, 65, 126)","(9, 15, 4)","(221, 188, 92)","(220, 182, 145)","(136, 87, 55)","(159, 162, 162)","(239, 195, 135)","(95, 54, 104)","(122, 19, 157)"
1,"(227, 177, 187)","(210, 105, 92)","(199, 140, 26)","(39, 39, 44)","(205, 200, 197)","(157, 142, 136)","(245, 118, 101)","(226, 127, 14)","(43, 31, 19)","(49, 47, 47)",...,"(163, 167, 144)","(169, 101, 57)","(192, 206, 152)","(62, 53, 53)","(140, 89, 50)","(211, 179, 147)","(70, 76, 42)","(185, 131, 78)","(114, 97, 223)","(190, 103, 203)"
2,"(236, 218, 221)","(53, 23, 14)","(237, 189, 85)","(169, 155, 144)","(50, 35, 30)","(60, 42, 36)","(198, 59, 56)","(176, 166, 141)","(157, 130, 60)","(209, 209, 208)",...,"(29, 59, 50)","(12, 31, 57)","(81, 105, 30)","(239, 229, 215)","(166, 175, 186)","(77, 43, 24)","(213, 218, 210)","(117, 66, 30)","(220, 186, 152)","(45, 22, 53)"
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_per.head(10)

,mallows,peach2,lamp,mos,pipe,coc,peach,flowers,ic,raccoon,...,6bf7bb6ba078ffb96fa1bf7dc73d28a0,fe7e1edbcda543fc2d7e3dc4bf54c5fe,d072cb092a6b5e3e5e9746ff42a928e3,68788d484729dc0297da343a0afe835e,ac144eb4fe749a4c231bbe874f0dd1ea,a5231680454b32cf7c34d19d7f55bf91,68320e05c6b538567d454c1bfd21f143,28c848151a54b63dd07543b3fcb346ae,350ecf9416b807877ec2acf132e6df28,96a5bda36aee92754d0a50ca2e4024ab
0,12.0,34.0,66.0,30.0,33.0,70.0,22.0,34.0,49.0,31.0,...,42.0,79.0,59.0,53.0,22.0,35.0,63.0,38.0,49.0,46.0
1,50.0,34.0,10.0,51.0,48.0,15.0,28.0,42.0,13.0,42.0,...,27.0,5.0,14.0,26.0,41.0,20.0,16.0,36.0,37.0,24.0
2,38.0,33.0,24.0,19.0,20.0,15.0,50.0,24.0,37.0,27.0,...,31.0,16.0,27.0,21.0,37.0,45.0,21.0,26.0,14.0,30.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Extract the two maximum values per column

(Maybe we use the maximum or the second maximum)

In [5]:
first_max = []
second_max = []
imgs_name = []
for i in df_col.keys():
    imgs_name = imgs_name + [i]
    maxs = df_per[i].nlargest(2).keys()
    first_max = first_max + [df_col[i][maxs[0]]]
    second_max = second_max + [df_col[i][maxs[1]]]

In [6]:
from ast import literal_eval as make_tuple

first_max = list(map(make_tuple, first_max))
second_max = list(map(make_tuple, second_max))

## Color sorting and color stripe image

Once determined the principal color(s) for each picture, now it's the time to sort them. There are many ways to do that, but I will focus in the RGB ans HSV color spaces. 

The sorting method depends mainly on the dataset, and to choose the best one it's useful to look at the color stripes for each method generated by the *color_stripe* function.

In [7]:
from color_analysis import color_stripe, SortingColors

### RGB sorting

The most simple way of sorting. First, it sorts the first component (R: red), then the second (G: green) and finally the third (B: blue) of the principal color of the image.

In [8]:
RGB = SortingColors((imgs_name, first_max), 1)
color_stripe([20,100], RGB.sorted_RGBvalues, 'ColorStripe_RGB')
RGB.sorted[0:10]

[('3eb5ddb4aebd19816bb664b28b9df0fc', (2, 5, 17)),
 ('edc9df272ba8f3408a6e0b8adca77aab', (3, 3, 3)),
 ('bjork2', (4, 4, 5)),
 ('lea', (6, 49, 47)),
 ('fe7e1edbcda543fc2d7e3dc4bf54c5fe', (7, 65, 126)),
 ('0e11868a7e6a459af2f92249a04a138c', (8, 35, 9)),
 ('umbrella', (9, 12, 36)),
 ('d072cb092a6b5e3e5e9746ff42a928e3', (9, 15, 4)),
 ('matcha', (10, 12, 4)),
 ('lea2', (13, 30, 22))]

### HSV sorting

The HSV color space represents the similarity between colors better than the RGB space. It uses hue, saturation and value as parameters.

In [9]:
def hsv(x): 
    return(colorsys.rgb_to_hsv(*x))

In [10]:
hsv_t = [hsv(i) for i in first_max]
HSV = SortingColors((imgs_name, first_max, hsv_t), 2)
color_stripe([20,100], HSV.sorted_RGBvalues, 'ColorStripe_HSV')
HSV.sorted[0:10]

[('edc9df272ba8f3408a6e0b8adca77aab', (3, 3, 3), (0.0, 0.0, 3)),
 ('09ebb694f9c66991467684098da547c2', (232, 232, 232), (0.0, 0.0, 232)),
 ('raccoon', (49, 47, 47), (0.0, 0.04081632653061224, 49)),
 ('plumas', (142, 44, 43), (0.0016835016835016758, 0.6971830985915493, 142)),
 ('kies', (206, 48, 46), (0.002083333333333326, 0.7766990291262136, 206)),
 ('eb3720c19c9058aa321df37e5c82b67a',
  (101, 24, 23),
  (0.002136752136752129, 0.7722772277227723, 101)),
 ('peach', (198, 59, 56), (0.0035211267605633756, 0.7171717171717171, 198)),
 ('agra', (67, 13, 10), (0.008771929824561412, 0.8507462686567164, 67)),
 ('2c27fde2ab88cfa75ff0ec8af468c03a',
  (22, 5, 4),
  (0.009259259259259264, 0.8181818181818182, 22)),
 ('peach2', (143, 51, 45), (0.010204081632653053, 0.6853146853146853, 143))]

### HLS sorting

HLS is similar to HSV, but instead the color space is organized by hue, saturation and lightness

In [11]:
def hls(x):
    return(colorsys.rgb_to_hls(*x))

In [12]:
hls_t = [hls(i) for i in first_max]
HLS = SortingColors((imgs_name, first_max, hls_t), 2)
color_stripe([20,100], HLS.sorted_RGBvalues, 'ColorStripe_HLS')
HLS.sorted[0:10]

[('edc9df272ba8f3408a6e0b8adca77aab', (3, 3, 3), (0.0, 3.0, 0.0)),
 ('raccoon', (49, 47, 47), (0.0, 48.0, -0.02127659574468085)),
 ('09ebb694f9c66991467684098da547c2', (232, 232, 232), (0.0, 232.0, 0.0)),
 ('plumas', (142, 44, 43), (0.0016835016835016758, 92.5, -0.5409836065573771)),
 ('kies', (206, 48, 46), (0.002083333333333326, 126.0, -0.64)),
 ('eb3720c19c9058aa321df37e5c82b67a',
  (101, 24, 23),
  (0.002136752136752129, 62.0, -0.639344262295082)),
 ('peach', (198, 59, 56), (0.0035211267605633756, 127.0, -0.5634920634920635)),
 ('agra', (67, 13, 10), (0.008771929824561412, 38.5, -0.76)),
 ('2c27fde2ab88cfa75ff0ec8af468c03a',
  (22, 5, 4),
  (0.009259259259259264, 13.0, -0.75)),
 ('peach2', (143, 51, 45), (0.010204081632653053, 94.0, -0.5268817204301075))]

### Luminosity sorting

Sorts by the percieved luminosity of the color

In [13]:
def lum (r,g,b):
    return math.sqrt( .241 * r + .691 * g + .068 * b )

In [14]:
lum_t = [lum(*i) for i in first_max]
LUM = SortingColors((imgs_name, first_max, lum_t), 2)
color_stripe([20,100], LUM.sorted_RGBvalues, 'ColorStripe_LUM')
LUM.sorted[0:10]

[('edc9df272ba8f3408a6e0b8adca77aab', (3, 3, 3), 1.7320508075688772),
 ('bjork2', (4, 4, 5), 2.0169283576765933),
 ('3eb5ddb4aebd19816bb664b28b9df0fc', (2, 5, 17), 2.256767599909215),
 ('2c27fde2ab88cfa75ff0ec8af468c03a', (22, 5, 4), 3.0048294460751013),
 ('matcha', (10, 12, 4), 3.3127028239792353),
 ('e74d123b69763dd689bc899ca04cc174', (17, 8, 38), 3.4941379480495613),
 ('d072cb092a6b5e3e5e9746ff42a928e3', (9, 15, 4), 3.5785471912495437),
 ('umbrella', (9, 12, 36), 3.5929096843644706),
 ('4cce5f10e307d2c1e501938389db9fa0', (22, 9, 24), 3.6267064948793415),
 ('afd05affb8ee9a16f8cd55ac30cfd969', (16, 17, 30), 4.200357127673788)]

### Step sorting

Based on the HSV color space, but also sorts by luminosity, generating a number of color segments. The ideal number of segments varies from dataset to dataset. To add continuity to the segments, the second step method inverts the luminosity every two segments.

In [15]:
def step (r,g,b, repetitions=1):
    
    lum = math.sqrt( .241 * r + .691 * g + .068 * b )
    h, s, v = colorsys.rgb_to_hsv(r,g,b)
    
    h2 = int(h * repetitions)
    lum2 = int(lum * repetitions)
    v2 = int(v * repetitions)
    
    return (h2, lum, v2)

In [16]:
for number_steps in range(0,20):
    step_t = [step(*i, number_steps) for i in first_max]
    STEP = SortingColors((imgs_name, first_max, step_t), 2)
    color_stripe([20,100], STEP.sorted_RGBvalues, 'ColorStripe_STEP' + str(number_steps))

In [17]:
def step_lum (r,g,b, repetitions=1):
    
    lum = math.sqrt( .241 * r + .691 * g + .068 * b )
    h, s, v = colorsys.rgb_to_hsv(r,g,b)
    
    h2 = int(h * repetitions)
    lum2 = int(lum * repetitions)
    v2 = int(v * repetitions)
    
    if h2 % 2 == 1:
        v2 = repetitions - v2
        lum = repetitions - lum
        
    return (h2, lum, v2)


In [18]:
for number_steps in range(0,20):
    steplum_t = [step_lum(*i, number_steps) for i in first_max]
    STEP_LUM = SortingColors((imgs_name, first_max, steplum_t), 2)
    color_stripe([20,100], STEP_LUM.sorted_RGBvalues, 'ColorStripe_STEPLUM' + str(number_steps))

## Plot the sorted images for a better perception of the images' order and save the order in a csv file

First, select the best sorting option for the dataset. In this case, I think the best one is the step sorting with luminosity using seven steps.

In [19]:
final_t = [step_lum(*i, 7) for i in first_max]
FINAL = SortingColors((imgs_name, first_max, final_t), 2)

In [20]:
from color_analysis import collage

In [21]:
collage(FINAL.sorted_names, 400, 4, "Imgs_done", "3clusters")

In [22]:
final_df = pd.DataFrame(list(zip(FINAL.sorted_names, FINAL.sorted_RGBvalues)), columns = ['sorted_names', 'sorted_RGBcolors'])
final_df.to_csv('collage/3clusters.csv', index = False)